### Updates `openalex.works.authors_and_affiliations` in Walden End to End workflow

### Step 1: Create enriched authors

In [0]:
%run ../utils/variables

In [0]:
CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_updates')
CLUSTER BY (work_id) AS (
WITH base_works AS (
    -- Read raw authorships from openalex_works_base (incremental)
    SELECT
        id AS work_id,
        authorships,
        updated_date AS updated_datetime
    FROM identifier('openalex' || :env_suffix || '.works.openalex_works_base')
    WHERE updated_date > max_updated_date
      AND authorships IS NOT NULL
      AND SIZE(authorships) > 0
),
institution_lineage AS (
    SELECT
        institution_id,
        COLLECT_LIST(ancestor_id) AS lineage_ids
    FROM openalex.mid.institution_ancestors
    WHERE NOT ARRAY_CONTAINS(SUPER_SYSTEM_INSTITUTIONS, ancestor_id)
    GROUP BY institution_id
),
author_institutions_with_details AS (
    SELECT
        ai.work_id,
        ai.author_sequence,
        ARRAY_DISTINCT(FLATTEN(COLLECT_SET(ai.raw_countries))) AS raw_parsed_countries,
        COLLECT_SET(
            STRUCT(
                inst.iso3166_code AS country_code,
                inst.display_name,
                CONCAT('https://openalex.org/I', ai.institution_id) AS id,
                ARRAY_SORT(
                    TRANSFORM(
                        ARRAY_COMPACT(CONCAT(ARRAY(ai.institution_id), COALESCE(il.lineage_ids, ARRAY()))), 
                        id -> CONCAT('https://openalex.org/I', id)
                    )
                ) AS lineage,
                CASE 
                    WHEN inst.ror_id IS NULL THEN NULL
                    WHEN inst.ror_id LIKE 'https://ror.org/%' THEN inst.ror_id
                    ELSE CONCAT('https://ror.org/', inst.ror_id) 
                END AS ror,
                inst.type
            )
        ) AS institutions
    FROM identifier('openalex' || :env_suffix || '.authors.author_institutions') ai
    LEFT JOIN openalex.institutions.institutions inst ON inst.id = ai.institution_id
    LEFT JOIN institution_lineage il USING (institution_id)
    WHERE ai.institution_id IS NOT NULL
    GROUP BY ai.work_id, ai.author_sequence
),
-- Map raw_affiliation_string -> institution_ids per (work_id, author_sequence)
affiliations_map_ids AS (
    SELECT
        work_id,
        author_sequence,
        raw_affiliation_string,
        ARRAY_DISTINCT(
            ARRAY_COMPACT(
                COLLECT_LIST(CONCAT('https://openalex.org/I', institution_id))
            )
        ) AS institution_ids
    FROM identifier('openalex' || :env_suffix || '.authors.author_institutions')
    WHERE institution_id IS NOT NULL
      AND raw_affiliation_string IS NOT NULL
    GROUP BY work_id, author_sequence, raw_affiliation_string
),
affiliations_map AS (
    SELECT
        work_id,
        author_sequence,
        MAP_FROM_ENTRIES(
            COLLECT_LIST(NAMED_STRUCT('key', raw_affiliation_string, 'value', institution_ids))
        ) AS aff_map
    FROM affiliations_map_ids
    GROUP BY work_id, author_sequence
),
-- Pre-aggregate institution details per work_id as a map[author_sequence -> details]
author_institution_lookup AS (
    SELECT
        work_id,
        MAP_FROM_ENTRIES(
            COLLECT_LIST(
                STRUCT(
                    author_sequence,
                    STRUCT(
                        institutions,
                        raw_parsed_countries,
                        aff_map
                    )
                )
            )
        ) AS author_lookup
    FROM (
        SELECT
            aid.work_id,
            aid.author_sequence,
            aid.institutions,
            aid.raw_parsed_countries,
            am.aff_map
        FROM author_institutions_with_details aid
        LEFT JOIN affiliations_map am 
            ON aid.work_id = am.work_id 
            AND aid.author_sequence = am.author_sequence
    )
    GROUP BY work_id
),
-- Explode authorships to join with parsed names lookup
exploded_for_parsed_names AS (
    SELECT
        work_id,
        updated_datetime,
        POSEXPLODE(authorships) AS (author_idx, authorship)
    FROM base_works
),
-- Join with parsed names lookup
with_parsed_names AS (
    SELECT
        e.work_id,
        e.updated_datetime,
        e.author_idx,
        e.authorship,
        pn.parsed_name
    FROM exploded_for_parsed_names e
    LEFT JOIN identifier('openalex' || :env_suffix || '.authors.parsed_names_lookup') pn
        ON TRIM(e.authorship.raw_author_name) = pn.raw_author_name
),
-- Reassemble authorships with parsed names
authorships_with_parsed_names AS (
    SELECT
        work_id,
        updated_datetime,
        TRANSFORM(
            ARRAY_SORT(
                COLLECT_LIST(
                    STRUCT(
                        author_idx,
                        STRUCT(
                            authorship.affiliations AS affiliations,
                            authorship.author AS author,
                            authorship.author_position AS author_position,
                            authorship.author_order_number AS author_order_number,
                            authorship.countries AS countries,
                            authorship.institutions AS institutions,
                            authorship.is_corresponding AS is_corresponding,
                            authorship.raw_affiliation_strings AS raw_affiliation_strings,
                            authorship.raw_author_name AS raw_author_name,
                            parsed_name AS parsed_name
                        ) AS authorship
                    )
                ),
                (left, right) -> CASE
                    WHEN left.author_idx < right.author_idx THEN -1
                    WHEN left.author_idx > right.author_idx THEN 1
                    ELSE 0
                END
            ),
            x -> x.authorship
        ) AS authorships
    FROM with_parsed_names
    GROUP BY work_id, updated_datetime
)
-- Final enrichment: add institution details, countries, affiliations mapping
SELECT
    ba.work_id,
    ba.updated_datetime,
    TRANSFORM(
        ba.authorships,
        (auth, idx) -> STRUCT(
            -- affiliations: map raw_affiliation_strings to institution_ids
            TRANSFORM(
                COALESCE(auth.raw_affiliation_strings, ARRAY()),
                s -> STRUCT(
                    COALESCE(ELEMENT_AT(ELEMENT_AT(ail.author_lookup, idx).aff_map, s), ARRAY()) AS institution_ids,
                    s AS raw_affiliation_string
                )
            ) AS affiliations,
            -- Preserve author field (id still NULL, will be assigned later)
            auth.author,
            -- All other fields
            auth.author_position,
            auth.author_order_number,
            -- countries from institutions (fallback to raw parsed countries)
            CASE
                WHEN ELEMENT_AT(ail.author_lookup, idx).institutions IS NOT NULL 
                     AND SIZE(FILTER(ELEMENT_AT(ail.author_lookup, idx).institutions.country_code, c -> c IS NOT NULL AND c <> '')) > 0
                    THEN ARRAY_SORT(ARRAY_DISTINCT(FILTER(ELEMENT_AT(ail.author_lookup, idx).institutions.country_code, c -> c IS NOT NULL AND c <> '')))
                WHEN ELEMENT_AT(ail.author_lookup, idx).raw_parsed_countries IS NOT NULL
                    THEN ELEMENT_AT(ail.author_lookup, idx).raw_parsed_countries
                ELSE ARRAY()
            END AS countries,
            -- institutions with full details
            COALESCE(ELEMENT_AT(ail.author_lookup, idx).institutions, ARRAY()) AS institutions,
            auth.is_corresponding,
            auth.raw_affiliation_strings,
            auth.raw_author_name,
            auth.parsed_name
        )
    ) AS authorships
FROM authorships_with_parsed_names ba
LEFT JOIN author_institution_lookup ail ON ba.work_id = ail.work_id);

### Step 2: Merge enriched updates into final `authors_and_affiliations` table

In [0]:
MERGE INTO identifier('openalex' || :env_suffix || '.works.authors_and_affiliations') AS target
USING identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_updates') AS source
ON target.work_id = source.work_id
WHEN MATCHED THEN UPDATE SET
  target.authorships = source.authorships,
  target.updated_datetime = source.updated_datetime
WHEN NOT MATCHED THEN INSERT (work_id, authorships, updated_datetime)
VALUES (source.work_id, source.authorships, source.updated_datetime);